In [ ]:
import re
import gc
import pandas as pd
import numpy as np
from nltk.tokenize import sent_tokenize, word_tokenize
from keras.preprocessing.text import Tokenizer,text_to_word_sequence
import nltk
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, GRU
from tensorflow.python.ops.rnn import bidirectional_dynamic_rnn
from tensorflow.compat.v1.nn.rnn_cell import GRUCell
from keras.engine.topology import Layer
from keras.models import Model
# nltk.download('punkt')
from tqdm import tqdm_notebook

In [ ]:
maxlen = 80
max_sentences = 15
max_words = 20000
embedding_dim = 100
batch_size = 256
validation_split = 0.2
hidden_size=150 
attention_size = 50
keep_prob1 = 0.8
keep_prob2 = 0.8

In [ ]:
def clean_text(text):
  '''
  Remove non-ascii characters, multiple spaces, and newlines
  '''
  text = re.sub(r'[^\x00-\x7f]', r'', text)
  text = re.sub(r'\n',' ', text)
  text = re.sub(r" +"," ",text)
  return text

In [ ]:
import os
os.listdir('../input/')

In [ ]:
data = pd.read_csv('../input/wine-reviews/winemag-data_first150k.csv')
data = data[data.duplicated('description', keep=False)]
data = data[:20000]

outlier_cutoff = np.percentile(data['points'], 95)
c_vec = np.clip(data['points'], 0, outlier_cutoff)
median_score = np.median(c_vec)
std_score = data['points'].std()
print(outlier_cutoff, median_score, std_score)
data['points'] = c_vec
data['points_zscore'] = ((c_vec-median_score)/std_score)

text = data['description']
para = text.apply(lambda x: x.split('.'))

merged_sent = []
for sentences in para:
    m = len(sentences)
    for sentence in sentences:
        merged_sent.append(sentence)
        
merged_sent = np.array(merged_sent)


X = []
y = []
reviews = []
for index, row in data.iterrows():
    X.append(row['description'])
    y.append(row['points_zscore'])

X = np.array(X)
y = np.array(y)

In [ ]:
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(merged_sent)

data = np.zeros((para.shape[0], max_sentences, maxlen), dtype='int32')
# print(reviews)
for i, review in enumerate(para):
    #print("in review")
    for j, sentence in enumerate(review):
        if j < max_sentences:
            tokens = text_to_word_sequence(sentence)
            k = 0
            for _, word in enumerate(tokens):
                if k < maxlen and tokenizer.word_index[word] < max_words:
                    data[i, j, k] = tokenizer.word_index[word]
                    k = k + 1

indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = y[indices]
nb_validation_samples = int(validation_split * data.shape[0])

x_train = data[:-nb_validation_samples]
y_train = y[:-nb_validation_samples]
x_val = data[-nb_validation_samples:]
y_val = y[-nb_validation_samples:]

In [ ]:
x_train.shape

In [ ]:
del X
del labels
gc.collect()

In [ ]:
embeddings_index = {}
f = open('../input/glove6b100dtxt/glove.6B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

word_index = tokenizer.word_index
embedding_matrix = np.random.random((len(word_index) + 1, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

del embeddings_index
gc.collect()

In [ ]:
def word_attention(inputs, att_size):
    inputs = tf.concat(inputs, 2) # (n, embedding_size, hiddensize) -> (n, 80, 300)
    hiddensize = inputs.shape[2].value  # D value - hidden size of the RNN layer -> 300
    
    # Trainable parameters
    W_word = tf.Variable(tf.random_normal([hiddensize, att_size], stddev=0.1)) # (hiddensize, attsize) -> (300, 50)
    b_word = tf.Variable(tf.random_normal([att_size], stddev=0.1)) # (1, 50)
    u_word = tf.Variable(tf.random_normal([att_size], stddev=0.1)) # (1, 50)
    v_word = tf.tanh(tf.tensordot(inputs, W_word, axes=1) + b_word)  # v = tanh(W.x + b) . (1,attsize) -> (n, 80, 50)
    vu_word = tf.tensordot(v_word, u_word, axes=1, name='vu')  # v.u (n,80)
    scores = tf.nn.softmax(vu_word)   # attention scores (n,80)

    output = tf.reduce_sum(inputs * tf.expand_dims(scores, -1), 1)
    return output

def sentence_attention(inputs, att_size):
    inputs = tf.concat(inputs, 2) # (n, embedding_size, hiddensize) -> (n, 80, 300)
    hiddensize = inputs.shape[2].value  # D value - hidden size of the RNN layer -> 300
    
    # Trainable parameters
    W_sent = tf.Variable(tf.random_normal([hiddensize, att_size], stddev=0.1)) # (hiddensize, attsize) -> (300, 50)
    b_sent = tf.Variable(tf.random_normal([att_size], stddev=0.1)) # (1, 50)
    u_sent = tf.Variable(tf.random_normal([att_size], stddev=0.1)) # (1, 50)
    v_sent = tf.tanh(tf.tensordot(inputs, W_sent, axes=1) + b_sent)  # v = tanh(W.x + b) . (1,attsize) -> (n, 80, 50)
    vu_sent = tf.tensordot(v_sent, u_sent, axes=1, name='vu')  # v.u (n,80)
    scores = tf.nn.softmax(vu_sent)   # attention scores (n,80)

    output = tf.reduce_sum(inputs * tf.expand_dims(scores, -1), 1)
    return output

In [ ]:
tf.reset_default_graph()

sentence_input = tf.placeholder(tf.int32, [None,max_sentences, maxlen])
input_len = tf.placeholder(tf.int32)
output_x = tf.placeholder(tf.int32, [None])
output_x = tf.cast(output_x,tf.float32) 

# Word level

with tf.variable_scope("word") as scope:
  count = 0
  mat = []
  for word in tf.unstack(sentence_input,axis=1):
    embeddings_var = tf.Variable(embedding_matrix, trainable=True)
    embeddings = tf.nn.embedding_lookup(embeddings_var, word, partition_strategy='div')
    embeddings = tf.cast(embeddings,tf.float32) 
    rnn_outputs, _ = bidirectional_dynamic_rnn(GRUCell(hidden_size, dtype=tf.float32), GRUCell(hidden_size, dtype=tf.float32), inputs=embeddings, dtype=tf.float32)
    weighted_inputs = word_attention(rnn_outputs,attention_size)
    weighted_inputs = tf.reshape(weighted_inputs, [input_len, 1, weighted_inputs.shape[1]])
    scope.reuse_variables()    
    mat.append(weighted_inputs)
    count+=1

after_word_attention = tf.stack(mat, axis=1)
after_word_attention = tf.reshape(after_word_attention, [input_len, 15, 300])
dropout_1 = tf.nn.dropout(after_word_attention, keep_prob1)
rnn_outputs_sent, _ = bidirectional_dynamic_rnn(GRUCell(hidden_size*1.5, dtype=tf.float32), GRUCell(hidden_size*1.5, dtype=tf.float32), inputs=dropout_1, dtype=tf.float32)
weighted_inputs_sent = sentence_attention(rnn_outputs_sent,attention_size)
dropout_2 = tf.nn.dropout(weighted_inputs_sent, keep_prob2)
fc = tf.keras.layers.Dense(units=1, activation='relu')(dropout_2)
#loss = tf.nn.sigmoid_cross_entropy_with_logits(logits=fc, labels=output_x)
loss = tf.reduce_mean(tf.keras.losses.MSE(fc,output_x))
optimizer = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(loss)

pred = tf.round(fc)
# n_correct = tf.equal(pred, output_x)
# accuracy = tf.reduce_mean(tf.cast(n_correct, tf.float32))
accuracy = tf.reduce_mean(tf.keras.losses.MAE(pred,output_x))


In [ ]:
def batch_generator(X, y, batchsize):
    size = X.shape[0]
    x_copy = X.copy()
    y_copy = y.copy()
    ind = np.arange(size)
    np.random.shuffle(ind)
    x_copy = x_copy[ind]
    y_copy = y_copy[ind]
    i = 0
    while True:
        if i + batchsize <= size:
            yield x_copy[i:i + batchsize], y_copy[i:i + batchsize]
            i += batchsize
        else:
            i = 0
            ind = np.arange(size)
            np.random.shuffle(ind)
            x_copy = x_copy[ind]
            y_copy = y_copy[ind]
            continue

In [ ]:
train_batch_generator = batch_generator(x_train, y_train, batch_size)
test_batch_generator = batch_generator(x_val, y_val, batch_size)

In [ ]:
x_val.shape

In [ ]:
with tf.Session() as sess:
    tf.initialize_all_variables().run()

    # Training 
    epochs = 10
    for epoch in range(epochs):
        acc_sum = 0
        #     n_batches = 0 
        print("\nEpoch: ",epoch+1)
        #     for i in range(0,20000, 200):
        #       try:
        #         xpoo = x_train[i:i+200]
        #         ypoo = y_train[i:i+200]
        #       except:
        #         xpoo = x_train[i::]
        #         ypoo = y_train[i::]
        num_batches = x_train.shape[0] // batch_size
        for b in tqdm_notebook(range(num_batches)):
            x_batch_train, y_batch_train = next(train_batch_generator)
            length = len(y_batch_train)
            loss_acc,opt, acc = sess.run([loss,optimizer,accuracy], feed_dict={sentence_input:x_batch_train, output_x:y_batch_train, input_len: length})
            acc_sum += acc
        #       n_batches += 1
        #           if(b % batch_size == 0):
        #         print(loss_acc, end =" ")
        #         print(acc)
        # print(acc_sum, n_batches)
        print('==> MSE Error: ', acc_sum/num_batches)
  # Validation
        val_accs=[]
        num_batches = x_val.shape[0] // batch_size
        for b in tqdm_notebook(range(num_batches)):
            x_batch_test, y_batch_test = next(test_batch_generator)
            length = len(y_batch_test)
            val_accs.append(sess.run(accuracy, feed_dict={sentence_input:x_batch_test, output_x:y_batch_test, input_len: length}))

    print("==> Test Accuracy: ", val_accs)